## 🔍 Market Regime Detection
The various decisions and actions for profit-seeking in financial markets is driven largely by how those markets behave. When asset prices have steadily been in an upward trend for the past several months, for example, many would stay invested or even use leverage to capture more gains. On the other hand, when priecs rise along with higher interest rate, as in a couple years ago, perhaps it’s safer to opt for commodity-related investments.\
These distinct patterns of market behavior are known as market regimes, and they often shift because of changing government policy or broader macroeconomic conditions. Consequently, the effectiveness of different investments can also change, with impacts on returns, volatility, and how prices move over time. \
For this reason, it is vital to detect market regime shifts so that traders and investors can stay informed to make the best choice of quantitative strategy and portfolio allocation to better align with current environment and maximize returns.\
Below are some popular techniques and their applications.

In [1]:
# Import financial data fetching function from first project
import sys
import os

# Use current working directory as base (Jupyter-safe)
project1_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'Financial-Data-API-Research-Week1'))
sys.path.append(project1_path)

# Import the function
from main import fetch_daily_data

## 📏 Rule-based Method
Rule-based methods rely on explicit thresholds derived from indicators like volatility, momentum, or moving averages. These methods are simple, transparent, and effective for making real-time risk management decisions, such as adjusting position sizes or stop-loss distances.\
One classic practice is that traders maintain an exposure/position and stop-loss tightness that inversely correlates with volatility.

In [27]:
# Import and Setup
import pandas as pd
import numpy as np

In [20]:
# Load historical AAPL data
data = fetch_daily_data("AAPL", "203-01-01", "2025-06-27").sort_index(ascending = True)

# Calculate log return
data['logReturn'] = np.log(data['adjClose']).diff()

# Rolling 20-day volatility as risk regime proxy
data['volatility'] = data['logReturn'].rolling(window=20).std()

# Define regime: High if above median, else low
data['riskRegime'] = np.where(data['volatility'] > data['volatility'].median(), 'High', 'Low')

# Adjust position size as a % of normal size: lower when uncertain, higher when favorable
data['posSize (%)'] = np.where(data['riskRegime'] == 'High', 0.5, 1.0)

# Adjust stop-loss as a % of price: looser when uncertain, tighter when favorable
data['stopLoss (%)'] = np.where(data['riskRegime'] == 'High', 2 * data['volatility'], 1 * data['volatility'])

# Sort data in descending order of date
data = data.sort_index(ascending = False)

data.head()

,open,high,low,close,volume,adjClose,logReturn,volatility,riskRegime,posSize (%),stopLoss (%)
2025-06-27,201.890,203.22,200.0000,201.08,73188571,201.08,0.000398,0.010631,Low,1.0,0.010631
2025-06-26,201.430,202.64,199.4600,201.00,50799121,201.00,-0.002782,0.010647,Low,1.0,0.010647
2025-06-25,201.450,203.67,200.6201,201.56,39525730,201.56,0.006271,0.010626,Low,1.0,0.010626
2025-06-24,202.590,203.44,200.2000,200.30,54064033,200.30,-0.005973,0.011921,Low,1.0,0.011921
2025-06-23,201.625,202.30,198.9600,201.50,55814272,201.50,0.002484,0.013841,Low,1.0,0.013841


## 🫧 K-means Clustering
K-means clustering is an unsupervised learning method that groups similar periods in market data based on features like returns, volatility, and momentum. It allows segmenting historical periods into distinct market regimes, making it easier to assign optimal trading strategies.

Different strategies work better in different regimes:
Momentum strategies thrive in trending markets.\
Mean-reversion strategies work better in sideways markets.\
Volatility breakout strategies perform well in high-volatility environments.\
Example: Detect a high-volatility regime → switch from mean-reversion to momentum breakout.

In [1]:
# Import and Setup
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Load historical TSLA data
data = fetch_daily_data("TSLA", "203-01-01", "2025-06-27").sort_index(ascending = True)

# Calculate log return
data['logReturn'] = np.log(data['adjClose']).diff()

# Rolling 20-day volatility as risk regime proxy
data['volatility'] = data['logReturn'].rolling(window=20).std()

# Calculate momentum
data['Momentum'] = data['adjClose'].pct_change(20)

# Standardize each feature
features = data[['logReturn', 'volatility', 'Momentum']].dropna()
scaled = StandardScaler().fit_transform(features)

# Apply K-Means with k = 3
kmeans = KMeans(n_clusters=3, random_state=42)
data.loc[features.index, 'KMeansRegime'] = kmeans.fit_predict(scaled)

# # Assign strategy per cluster
# strategy_map = {0: 'Momentum', 1: 'Mean Reversion', 2: 'Neutral'}
# data['strategy'] = data['KMeansRegime'].map(strategy_map)

data.head()

NameError: name 'StandardScaler' is not defined

In [ ]:
cluster_means = features.copy()
cluster_means['cluster'] = kmeans.labels_
print(cluster_means.groupby('cluster').mean())
sns.pairplot(cluster_means, hue='cluster')

## 🕸 Hidden Markov Model (HMM)
HMM is a probabilistic model that assumes market regimes are hidden states generating observable returns and infers these latent states and their probabilities. It enables dynamic strategy switching and probabilistic risk adjustments.

on historical price data, classify regimes such as bull or bear markets. 
Fit on log returns and predict the most probable state at each time step opting for trend-following strategies in one state and mean-reversion strategies in another. 
switch strategies.

In [ ]:
# Import and Setup
from hmmlearn.hmm import GaussianHMM

In [ ]:
# Drop NaNs and use returns for HMM
returns = data['LogRet'].dropna().values.reshape(-1, 1)
hmm = GaussianHMM(n_components=2, covariance_type='full', n_iter=1000, random_state=42)
hmm.fit(returns)
hidden_states = hmm.predict(returns)

# Store regime and probability
data.loc[data['LogRet'].dropna().index, 'HMM_State'] = hidden_states
prob_matrix = hmm.predict_proba(returns)
data.loc[data['LogRet'].dropna().index, 'HMM_Prob_State0'] = prob_matrix[:, 0]

# Example logic: Use strategy & risk scaling based on HMM state
data['HMM_Strategy'] = np.where(data['HMM_State'] == 0, 'Trend', 'Reversion')
data['HMM_Position_Size'] = np.where(data['HMM_Prob_State0'] > 0.8, 1.0, 0.5)

## 📈 Visual Example: Strategy vs. Regimes

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(2):
    mask = data['HMM_State'] == i
    plt.plot(data.index[mask], data['Adj Close'][mask], '.', label=f'HMM Regime {i}')
plt.legend()
plt.title('HMM-Inferred Market Regimes')
plt.show()